Translation done in Translation.ipynb

In [ ]:
# predict data Path :
# data_path = '/export/home/cse200093/brat_data/Expe_TRANSL/expe_translation_Gold_after_conversion_train_test/test_eng_opus-FT_txt'
data_path = '/export/home/cse200093/brat_data/QUAERO_FrenchMed/corpus/test/EMEA_en_opus-FT_txt'
# data_path = '/export/home/cse200093/brat_data/n2c2_2019/test_restrict_proc_chem_devi_diso'
# data_path = '/export/home/cse200093/brat_data/n2c2_2019/train_restrict_proc_chem_devi_diso'
# Mantra/English
# data_path = '/export/home/cse200093/brat_data/Mantra_English'


# vocab path and cdb path
DATA_DIR = "/export/home/cse200093/MedCAT"
vocab_path = DATA_DIR + "/vocab.dat"
cdb_path = DATA_DIR + "/cdb-umls-v1.dat"

# res_path = "/export/home/cse200093/Expe_Translation/results/MedCAT_Gold_test_en_opus-FT"
res_path = '/export/home/cse200093/Expe_Translation/results/MedCAT_EMEA_test_en_opus-FT'
# res_path = '/export/home/cse200093/Expe_Translation/results/MedCAT_n2c2_2019_test'
# res_path = '/export/home/cse200093/Expe_Translation/results/MedCAT_n2c2_2019_train'
# res_path = '/export/home/cse200093/Expe_Translation/results/MedCAT_Mantra_English'

In [ ]:
# gold standard path
# stand_path = '/export/home/cse200093/brat_data/Expe_TRANSL/expe_translation_Gold_after_conversion_train_test/test/'
stand_path = '/export/home/cse200093/brat_data/QUAERO_FrenchMed/corpus/test/EMEA/'
# stand_path = '/export/home/cse200093/brat_data/n2c2_2019/test_restrict_proc_chem_devi_diso/'
# stand_path = '/export/home/cse200093/brat_data/n2c2_2019/train_restrict_proc_chem_devi_diso/'
# stand_path = '/export/home/cse200093/brat_data/Mantra_English/'

In [ ]:
import glob
import os
from tqdm import tqdm
import shutil
from spacy import displacy
import gensim
from tokenizers import ByteLevelBPETokenizer
from gensim.models import Word2Vec

import pandas as pd
import numpy as np
import seaborn as sns
import json 
from matplotlib import pyplot as plt

from medcat.cat import CAT
from medcat.cdb import CDB
from medcat.config import Config
from medcat.vocab import Vocab
from medcat.meta_cat import MetaCAT
from medcat.config_meta_cat import ConfigMetaCAT
from medcat.preprocessing.tokenizers import TokenizerWrapperBPE, TokenizerWrapperBERT
from tokenizers import ByteLevelBPETokenizer

In [ ]:
# map a type code in umls to a semantic type
dic_typecode2type = {}
f = open('/export/home/cse200093/Expe_Translation/Axes_script/umls_group_info.txt', 'r')
for line in tqdm(f):
    line = line.rstrip('\n')
    ents = line.split('|')
    type_name = ents[0]
    type_name_spe = ents[-1]
    type_code = ents[2]
    dic_typecode2type[type_code] = type_name
f.close()

len(dic_typecode2type)

In [ ]:
# Build model with cdb and vocab

# Create and load the CDB (Concept Database)
cdb = CDB.load(cdb_path)
# Create and load the Vocabulary
vocab = Vocab.load(vocab_path)
# Setup config
config = Config()
config.general['spacy_model'] = 'en_core_web_md'
# Get the required tokenizer (note that we have already downloaded the required models)
mc = MetaCAT.load('/export/home/cse200093/Expe_Translation/Status')
# To add the meta-annotation model to the medcat pipeline
cat = CAT(cdb=cdb, config=config, vocab=vocab, meta_cats=[mc])

In [ ]:
# read predict data
os.chdir(data_path)
my_files = glob.glob('*.txt')
len(my_files)

In [ ]:
# read all txt files and put them in a dataframe
df_text = pd.DataFrame(columns=['file_name','text_english'])

for file_name in tqdm(my_files):
    f = open(data_path+"/"+file_name, "r")
    text = f.read()
    f.close()
    #print(text)
    df_text = df_text.append({'file_name':file_name, 'text_english':text}, ignore_index=True)
    
df_text

In [ ]:
# Anotate documents
# format the df_text to match: required input data for multiprocessing = [(doc_id, doc_text), (doc_id, doc_text), ...]
def data_iterator(data):
    for id, row in data[['text_english']].iterrows():
        yield (id, str(row['text_english']))

In [ ]:
# Set a batch size to control for the variablity between document sizes
batch_size_chars = 500000 # Batch size (BS) in number of characters

# Run model
results = cat.multiprocessing(data_iterator(df_text),  # Formatted data
                              batch_size_chars = batch_size_chars,
                              nproc=8) # Number of processors

In [ ]:
# save results
f = open(res_path, 'w')

for key in results:
    filename = my_files[key]
    f.write(filename+'\n')
    entities = results[key]['entities']
    for ent in entities:
        if len(entities[ent]['type_ids']) != 0:
            line = entities[ent]['pretty_name']+'\t'+entities[ent]['cui']+'\t'+entities[ent]['type_ids'][0]+'\t'+entities[ent]['types'][0]
        else:
            line = entities[ent]['pretty_name']+'\t'+entities[ent]['cui']+'\t'+'unknown_typeId'+'\t'+'unknown_type'

        f.write(line+'\n')
    f.write('\n')

f.close()

In [ ]:
keep_types = ['CHEM','DEVI','DISO','PROC']

In [ ]:
# function to read precedently saved results of MedCAT
def read_results_cui(path):
    # dictionary containing file name as keys and a list of cuis as values
    dic_file_ent = {}

    # read result files to fill dic
    f = open(path, 'r')
    #lines = [line.rstrip('\n') for line in f]
    key = None
    for line in tqdm(f):
        line = line.rstrip('\n')
        if line.endswith('.txt'):
            dic_file_ent[line[:-4]] = []
            key = line[:-4]
        elif line == '':
            continue
        else:
            ent = line.split('\t')
            cui = ent[1]
            type_code = ent[2]
#             dic_file_ent[key].append(cui)
            # filter 4 types
            if type_code != 'unknown_typeId' and type_code != 'unk' and dic_typecode2type[type_code] in keep_types:
                type_name = dic_typecode2type[type_code]
                dic_file_ent[key].append(cui)
            else:
                continue
    f.close()
    
    return dic_file_ent

In [ ]:
dic_file_ent_built = read_results_cui(res_path)
len(dic_file_ent_built)

In [ ]:
# read gold standard
os.chdir(stand_path)
my_files = glob.glob('*.ann')
len(my_files)

In [ ]:
# dictionary containing correct results
dic_file_ent_standard = {}

for file in tqdm(my_files):
    dic_file_ent_standard[file[:-4]] = []
    f = open(stand_path+file, 'r')
    for line in f:
        line = line.rstrip('\n')
        if line.startswith('T'):
            type_name = line.split('\t')[1].split(' ')[0]
        if line.startswith('#'):
            eles = line.split('\t')
            #print(eles)
            cui = eles[-1]
            # filter 4 types
            if type_name in keep_types:
                dic_file_ent_standard[file[:-4]].append(cui)
        else:
            continue

In [ ]:
def convert_list_to_dic(dic):
    for file_name in tqdm(dic):
        key_list = list(set(dic[file_name]))
        value_list = [dic[file_name].count(x) for x in key_list]
        res = dict(zip(key_list, value_list))
        dic[file_name] = res

In [ ]:
convert_list_to_dic(dic_file_ent_built)
convert_list_to_dic(dic_file_ent_standard)

In [ ]:
df_compare = pd.DataFrame(columns = ['file_name','correct_ents','built_model_ents'])

for file in tqdm(dic_file_ent_standard):
    df_compare = df_compare.append({'file_name':file, 'correct_ents':dic_file_ent_standard[file], 
                       'built_model_ents':dic_file_ent_built[file]},
                     ignore_index=True)
    
df_compare

In [ ]:
df_compare['built_TP'] = 0
df_compare['built_FP'] = 0
df_compare['built_FN'] = 0

df_compare['built_precision'] = 0
df_compare['built_recall'] = 0
df_compare['built_f1-score'] = 0

# calculate TP,FP,FN for each model
for index, row in tqdm(df_compare.iterrows()):
    for cui in row['built_model_ents']:
        if cui in row['correct_ents']:
            df_compare.loc[index,'built_TP'] += min(row['correct_ents'][cui], row['built_model_ents'][cui])
        else:
            df_compare.loc[index,'built_FP'] += row['built_model_ents'][cui]
    for cui in row['correct_ents']:
        if cui not in row['built_model_ents']:
            df_compare.loc[index,'built_FN'] += row['correct_ents'][cui]
            
            
# calculate precision and recall for each model
for index, row in tqdm(df_compare.iterrows()):
    if row['built_TP'] != 0:
        df_compare.loc[index,'built_precision'] = row['built_TP']/(row['built_TP']+row['built_FP'])

    if row['built_TP'] != 0:
        df_compare.loc[index,'built_recall'] = row['built_TP']/(row['built_TP']+row['built_FN'])

# calculate f1-score for each model
for index, row in tqdm(df_compare.iterrows()):
    if row['built_TP'] != 0:
        df_compare.loc[index,'built_f1-score'] = (2*row['built_precision']*row['built_recall'])/(row['built_precision']+row['built_recall'])
            

df_compare

In [ ]:
# calculate mean precision, mean recall, mean f1-score

built_avg_precision = np.mean(df_compare['built_precision'])
built_avg_recall = np.mean(df_compare['built_recall'])
built_avg_f1_score = np.mean(df_compare['built_f1-score'])

print(f'Average precision of the built model: {built_avg_precision}')
print(f'Average recall of the built model: {built_avg_recall}')
print(f'Average f1-score of the built model: {built_avg_f1_score}')

In [ ]:
# save df_compare for boostrap.ipynb
df_compare.to_pickle('/export/home/cse200093/Expe_Translation/Axes_script/EMEA_res_axis3.pkl')